In [1]:
{-# LANGUAGE OverloadedStrings #-}

-- :!if [ -d solarized/ ]; then rm -rf solarized; fi
-- :!git clone https://github.com/altercation/solarized
:!cd solarized
:!ls .git/objects/pack

pack-44abe4e456180e053619e0422cb92be7fe650af5.idx
pack-44abe4e456180e053619e0422cb92be7fe650af5.pack

In [2]:
import System.Directory (getDirectoryContents)
import System.FilePath  ((</>), takeExtension)

getPackIndices :: FilePath -> IO [FilePath]
getPackIndices path = do
    let packPath = path </> "objects" </> "pack"
    files <- getDirectoryContents packPath
    let indices = filter (\f -> takeExtension f == ".idx") files
    return $ map (packPath </>) indices

getPackIndices ".git"

[".git/objects/pack/pack-44abe4e456180e053619e0422cb92be7fe650af5.idx"]

In [3]:
import           Data.ByteString      (ByteString)
import qualified Data.ByteString as B

packIndex <- B.readFile . head =<< getPackIndices ".git"

In [4]:
import           Data.Attoparsec.ByteString            (Parser)
import qualified Data.Attoparsec.ByteString.Char8 as A
import qualified Data.Attoparsec.ByteString       as A (word8)

parsed :: Parser a -> ByteString -> a
parsed parser = either error id . A.parseOnly parser

In [5]:
import Data.Word (Word8)

parsePackIndexHeader :: Parser [Word8]
parsePackIndexHeader = do
    traverse A.word8 start
    traverse A.word8 version
    where
        start   = [255, 116, 79, 99]
        version = [0, 0, 0, 2]

parsed parsePackIndexHeader packIndex

[0,0,0,2]